In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [ ]:
import openai
import pandas as pd

In [ ]:
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [ ]:
prompt = f'''
پاسخ دو مدل زبانی به یک سوال پزشکی به شما داده خواهد شد وظیفه شما این است که مطابق مراحل ارزیابی و معیار های ارزیابی داوری کنید که
الف: پاسخ اول بهتر است
ب: پاسخ دوم بهتر است

لطفا از بین این سه گزینه انتخاب کنید و از ارائه اطلاعات اضافی خودداری کنید

مراحل ارزیابی:
۱.پرسش را بخوانید
۲.پاسخ پزشک به این پرسش را بخوانید
۳.پاسخ مدل زبانی نسخت به این پرسش را بخوانید
۴.پاسخ مدل زبانی دوم به این پرسش را بخوانید
۵.مطابق معیار های ارزیابی داوری خود را انجام دهید

معیار های ارزیابی:
۱.ارائه پاسخ جامع تر
۲. عدم وجود اطلاعات نادرست

## پرسش
{{question}}\n

## پاسخ پزشک
{{dresp}}\n

## پاسخ مدل زبانی نخست
{{lm1resp}}\n

## پاسخ مدل زبانی دوم
{{lm2resp}}\n
    '''

In [ ]:
def append_record_to_excel(file_path,question,answer,Sources,prmpt,opponent_answer,gao_answer,judgement):
    new_record = {
        'Question': question,
        'answer': answer,
        'Sources': Sources,
        'Prompt':prmpt,
        'pipeline_answer': opponent_answer,
        'gao_answer': gao_answer,
        'judgement':judgement,
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)


In [ ]:
def get_gpt4o_answer(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0
    )
    result = response['choices'][0]['message']['content'].strip().lower()
    return result

In [ ]:
df = pd.read_excel('answers.xlsx')
i = 0
bgn = 0
for index,row in df.iterrows():
        question = row['Question']
        answer = row['answer']
        Sources = row['Sources']
        pipe_answer = row['Meerkat8b-aya(translator)_answer']
        gao_answer = row['gao_answer']
        prmpt = prompt.format(question=question,dresp=answer,lm1resp=pipe_answer,lm2resp=gao_answer)
        i += 1
        if i < bgn:
           continue
        judgement = get_gpt4o_answer(prmpt)
        file_path = 'result.xlsx'
        append_record_to_excel(file_path,question,answer,Sources,prmpt,pipe_answer,gao_answer,judgement)
        print(i)
        print(judgement)
        print('============================================================')

1978
ب: پاسخ دوم بهتر است
1979
ب: پاسخ دوم بهتر است
1980
ب: پاسخ دوم بهتر است
1981
الف: پاسخ اول بهتر است
1982
ب: پاسخ دوم بهتر است
1983
ب: پاسخ دوم بهتر است
1984
ب: پاسخ دوم بهتر است
1985
ب: پاسخ دوم بهتر است
1986
ب: پاسخ دوم بهتر است
1987
الف: پاسخ اول بهتر است
1988
ب: پاسخ دوم بهتر است
1989
الف: پاسخ اول بهتر است
1990
ب: پاسخ دوم بهتر است
1991
ب: پاسخ دوم بهتر است
1992
ب: پاسخ دوم بهتر است
1993
ب: پاسخ دوم بهتر است
1994
الف: پاسخ اول بهتر است
1995
**الف: پاسخ اول بهتر است**
1996
ب: پاسخ دوم بهتر است
1997
ب: پاسخ دوم بهتر است
1998
ب: پاسخ دوم بهتر است
1999
ب: پاسخ دوم بهتر است
2000
الف: پاسخ اول بهتر است


In [ ]:
df = pd.read_excel('result.xlsx')
i = 0
pipe_answer_prefered = 0
gaos_answer_prefered = 0
for index,row in df.iterrows():
    judgement = row['judgement']
    if 'اول' in judgement:
       pipe_answer_prefered += 1
    if 'دوم' in judgement:
       gaos_answer_prefered += 1

In [ ]:
print("in " + str(pipe_answer_prefered/20.0) +"% of question Meerkat8b-aya(translator)s answer prefered")
print("in " + str(gaos_answer_prefered/20.0) +"% of question gaokerenas answer prefered")

in 20.0% of question Meerkat8b-aya(translator)s answer prefered
in 80.0% of question gaokerenas answer prefered
